In [1]:
import os
import sys
from bs4 import BeautifulSoup as BS
from pathlib import Path

In [34]:
"""
Display warning banner on latest/'versionless' docs that aren't in current release.

Requires a single commandline argument for the latest version.
"""

# def tempname(file):
    # *name, ext = file.split('.')
    # return ('.'.join([*name, 'tmp', ext])) 

def adjust(root, file, dry = False):
    '''
    mark an html file deprecated
    dry: do not overwrite, but go to temp file.
    '''
    p = Path(root)
    # if ('.tmp' in Path(file).suffixes): return # prefer to ignore own temp files.
                
    with open(p / file, "rt") as f:
        parse = BS(f, 'lxml')
    
        s = ' '.join(file.split('.')[:-1])
        query = ' '.join(s.split('_')) # works well for (mpl_)examples, gallery
        
        if 'index' == s:
            try: query = parse.find('h1').text[:-1].replace('_', ' ')
            except Exception: query = ' '.join(p.parts)
            
            # ' '.join(p.parts) if s == 'index'
        search = '''https://matplotlib.org/search.html?q={}'''.format(query)
        
        for child in parse.html.children:
            # accessing children outside of iteration confuses beautifulsoup
            
            # if child.name == 'img':
                # WIP: attempt to deprecate image, if it isn't static
                # if os.path.exists(p / ):
                    # child['src'] = tempname(child['src'])

            if child.name == 'body':                
                # expect to share stylesheet with class="warning"
                banner = BS('''<div id="unreleased-message"> 
            You are reading documentation that no longer exists in the
            current release of Matplotlib! <a href="{}">Try searching for
            an updated version?</a>
            </div>'''.format(search), 'html.parser')
                
                inject = True
                for div in child.children:
                    if div.name == 'div' and div.attrs.get('id', 'none') \
                                                == 'unreleased-message':     
                        div.replaceWith(banner)
                        inject = False; break
                if inject:
                    child.insert(0, banner)

        if not dry:
            with open(p/file, "wt") as g:
                g.write(str(parse))
        else:
            print( '' if inject else '(rerun)', query )

In [36]:
adjust('examples/images_contours_and_fields', 'contourf_log.html', dry=False)
# adjust('devel', 'index.html', True)
# adjust('api/_as_gen', 'matplotlib.animation.Animation.save.html', True)

In [15]:
# these cleaned-out examples don't have a body, just a broken redirect.
! git rm examples/old_animation/* && git rm examples/pngsuite/*

rm 'examples/old_animation/animate_decay_tk_blit.html'
rm 'examples/old_animation/animation_blit_gtk.html'
rm 'examples/old_animation/animation_blit_gtk2.html'
rm 'examples/old_animation/animation_blit_qt4.html'
rm 'examples/old_animation/animation_blit_tk.html'
rm 'examples/old_animation/animation_blit_wx.html'
rm 'examples/old_animation/draggable_legend.html'
rm 'examples/old_animation/dynamic_collection.html'
rm 'examples/old_animation/dynamic_image_gtkagg.html'
rm 'examples/old_animation/dynamic_image_wxagg2.html'
rm 'examples/old_animation/gtk_timeout.html'
rm 'examples/old_animation/histogram_tkagg.html'
rm 'examples/old_animation/index.html'
rm 'examples/old_animation/movie_demo.html'
rm 'examples/old_animation/simple_anim_gtk.html'
rm 'examples/old_animation/simple_anim_tkagg.html'
rm 'examples/old_animation/simple_idle_wx.html'
rm 'examples/old_animation/simple_timer_wx.html'
rm 'examples/old_animation/strip_chart_demo.html'
rm 'examples/pngsuite/index.html'
rm 'examples/pngsu

In [17]:
latest = '2.2.2' # sys.argv[-1]
verbose = True
dry = False

for tree in ('examples', 'mpl_examples', 'plot_directive', 'gallery',
             'devel', 'users', 'api', 'faq', 'glossary'): # '_images'
    
    for root, dirs, files in os.walk(tree):
        # FIXME: how to run from another directory without introducing '..' to root?
        
        for file in files:
            if not os.path.exists(os.path.join(latest, root, file)):
                # if missing from latest release
                
                if file.endswith('.html'):
                    adjust(root, file, dry)
                elif file.endswith('.png'):
                    pass
                    # os.rename(file, tempname(file))
                    
                elif file.endswith('.py') or file.endswith('.pdf'):
                    os.system('git rm ' + os.path.join(root, file))
                
                # if verbose: print(os.path.join(root, file))
                    
            elif verbose:
                print('SKIP ' + os.path.join(root, file))

SKIP gallery/index.html
SKIP gallery/recipes/fill_between_alpha.html
SKIP gallery/recipes/common_date_problems.html
SKIP gallery/recipes/transparent_legends.html
SKIP gallery/recipes/placing_text_boxes.html
SKIP gallery/recipes/share_axis_lims_views.html
SKIP gallery/recipes/create_subplots.html
SKIP gallery/units/artist_tests.html
SKIP gallery/units/ellipse_with_units.html
SKIP gallery/units/annotate_with_units.html
SKIP gallery/units/bar_unit_demo.html
SKIP gallery/units/basic_units.html
SKIP gallery/units/bar_demo2.html
SKIP gallery/units/radian_demo.html
SKIP gallery/units/units_scatter.html
SKIP gallery/units/units_sample.html
SKIP gallery/units/evans_test.html
SKIP gallery/axisartist/demo_parasite_axes2.html
SKIP gallery/axisartist/demo_ticklabel_direction.html
SKIP gallery/axisartist/demo_parasite_axes.html
SKIP gallery/axisartist/simple_axisartist1.html
SKIP gallery/axisartist/simple_axisline3.html
SKIP gallery/axisartist/demo_axisline_style.html
SKIP gallery/axisartist/axis_di

SKIP gallery/animation/random_walk.html
SKIP gallery/animation/double_pendulum_sgskip.html
SKIP gallery/animation/animated_histogram.html
SKIP gallery/animation/strip_chart.html
SKIP gallery/animation/unchained.html
SKIP gallery/frontpage/contour.html
SKIP gallery/frontpage/membrane.html
SKIP gallery/frontpage/histogram.html
SKIP gallery/frontpage/3D.html
SKIP gallery/images_contours_and_fields/interpolation_methods.html
SKIP gallery/images_contours_and_fields/figimage_demo.html
SKIP gallery/images_contours_and_fields/image_nonuniform.html
SKIP gallery/images_contours_and_fields/shading_example.html
SKIP gallery/images_contours_and_fields/tripcolor_demo.html
SKIP gallery/images_contours_and_fields/trigradient_demo.html
SKIP gallery/images_contours_and_fields/contour_label_demo.html
SKIP gallery/images_contours_and_fields/custom_cmap.html
SKIP gallery/images_contours_and_fields/barb_demo.html
SKIP gallery/images_contours_and_fields/image_masked.html
SKIP gallery/images_contours_and_fiel

SKIP gallery/lines_bars_and_markers/gradient_bar.html
SKIP gallery/lines_bars_and_markers/xcorr_acorr_demo.html
SKIP gallery/lines_bars_and_markers/step_demo.html
SKIP gallery/lines_bars_and_markers/categorical_variables.html
SKIP gallery/style_sheets/bmh.html
SKIP gallery/style_sheets/plot_solarizedlight2.html
SKIP gallery/style_sheets/ggplot.html
SKIP gallery/style_sheets/fivethirtyeight.html
SKIP gallery/style_sheets/grayscale.html
SKIP gallery/style_sheets/style_sheets_reference.html
SKIP gallery/style_sheets/dark_background.html
SKIP gallery/ticks_and_spines/colorbar_tick_labelling_demo.html
SKIP gallery/ticks_and_spines/spines_dropped.html
SKIP gallery/ticks_and_spines/spines_bounds.html
SKIP gallery/ticks_and_spines/date_index_formatter.html
SKIP gallery/ticks_and_spines/custom_ticker1.html
SKIP gallery/ticks_and_spines/scalarformatter.html
SKIP gallery/ticks_and_spines/major_minor_demo.html
SKIP gallery/ticks_and_spines/spine_placement_demo.html
SKIP gallery/ticks_and_spines/ce

SKIP devel/license.html
SKIP devel/add_new_projection.html
SKIP devel/portable_code.html
SKIP devel/coding_guide.html
SKIP devel/testing.html
SKIP devel/gitwash/index.html
SKIP devel/gitwash/git_resources.html
SKIP devel/gitwash/git_development.html
SKIP devel/gitwash/development_workflow.html
SKIP devel/gitwash/maintainer_workflow.html
SKIP devel/gitwash/following_latest.html
SKIP devel/gitwash/patching.html
SKIP devel/gitwash/git_install.html
SKIP devel/gitwash/git_intro.html
SKIP devel/gitwash/configure_git.html
SKIP devel/gitwash/dot2_dot3.html
SKIP devel/gitwash/forking_hell.html
SKIP devel/gitwash/set_up_fork.html
SKIP devel/MEP/MEP24.html
SKIP devel/MEP/index.html
SKIP devel/MEP/MEP29.html
SKIP devel/MEP/MEP28-1.py
SKIP devel/MEP/MEP27.html
SKIP devel/MEP/MEP28.html
SKIP devel/MEP/MEP22.html
SKIP devel/MEP/README.html
SKIP devel/MEP/MEP21.html
SKIP devel/MEP/MEP13.html
SKIP devel/MEP/MEP11.html
SKIP devel/MEP/MEP10.html
SKIP devel/MEP/MEP09.html
SKIP devel/MEP/MEP25.html
SKIP de

SKIP api/_as_gen/matplotlib.artist.Artist.set_gid.html
SKIP api/_as_gen/matplotlib.axis.YTick.get_pad_pixels.html
SKIP api/_as_gen/matplotlib.pyplot.pcolor.html
SKIP api/_as_gen/matplotlib.axis.Axis.update_units.html
SKIP api/_as_gen/matplotlib.axis.XTick.properties.html
SKIP api/_as_gen/matplotlib.axis.Tick.have_units.html
SKIP api/_as_gen/matplotlib.axes.Axes.get_default_bbox_extra_artists.html
SKIP api/_as_gen/matplotlib.animation.Animation.html
SKIP api/_as_gen/matplotlib.axes.Axes.set_prop_cycle.html
SKIP api/_as_gen/matplotlib.axis.Tick.set_alpha.html
SKIP api/_as_gen/matplotlib.axis.XAxis.tick_top.html
SKIP api/_as_gen/matplotlib.axis.XTick.set_contains.html
SKIP api/_as_gen/matplotlib.axis.Axis.axes.html
SKIP api/_as_gen/matplotlib.artist.Artist.set.html
SKIP api/_as_gen/matplotlib.pyplot.cool.html
SKIP api/_as_gen/matplotlib.pyplot.angle_spectrum.html
SKIP api/_as_gen/matplotlib.axis.XAxis.OFFSETTEXTPAD.html
SKIP api/_as_gen/matplotlib.axis.XAxis.get_clip_path.html
SKIP api/_a

SKIP api/_as_gen/matplotlib.animation.ImageMagickWriter.html
SKIP api/_as_gen/matplotlib.axes.Axes.get_adjustable.html
SKIP api/_as_gen/matplotlib.axes.Axes.get_yminorticklabels.html
SKIP api/_as_gen/matplotlib.axis.Axis.get_minpos.html
SKIP api/_as_gen/matplotlib.gridspec.SubplotSpec.html
SKIP api/_as_gen/matplotlib.axis.YAxis.cla.html
SKIP api/_as_gen/mpl_toolkits.mplot3d.art3d.poly_collection_2d_to_3d.html
SKIP api/_as_gen/matplotlib.pyplot.cohere.html
SKIP api/_as_gen/matplotlib.axes.Axes.margins.html
SKIP api/_as_gen/matplotlib.axis.Axis.get_data_interval.html
SKIP api/_as_gen/matplotlib.axis.XTick.get_pad_pixels.html
SKIP api/_as_gen/matplotlib.axis.XAxis.set_default_intervals.html
SKIP api/_as_gen/matplotlib.axis.Axis.iter_ticks.html
SKIP api/_as_gen/matplotlib.axis.XAxis.get_transformed_clip_path_and_affine.html
SKIP api/_as_gen/matplotlib.axes.Axes.get_xaxis.html
SKIP api/_as_gen/matplotlib.axes.Axes.update_from.html
SKIP api/_as_gen/mpl_toolkits.mplot3d.art3d.Path3DCollection

Everyone else is deprecated.

In [ ]:
# WIP: remove whitespace changes
# re-ordering of tag attributes seems to be interfering.
# might have to use a slower but less strict parser.
# https://til.hashrocket.com/posts/696df00135-remove-whitespace-changes-then-git-add-p

# ! git diff -w origin/master | git apply --cached --ignore-whitespace
# ! git checkout -- && git reset && git add